Setup 

In [ ]:

import tensorflow as tf 
from tensorflow import keras 
from keras import layers 
from dataclasses import dataclass # có chức năg tạo các lớp dữ liệu 
import numpy as np 
import pandas as pd 
import glob
import re 
from pprint import pprint 


Set-up 

In [ ]:
@dataclass 
class Config:
    MAX_LEN = 256 
    BATCH_SIZE = 32 
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128 
    NUM_HEADS = 8
    FF_DIM = 128 
    NUM_LAYERS = 1 

config = Config()


Load the data

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
# Xây dựng hàm lấy dữ liệu văn bản từ các files tài liệu 
def get_text_list_from_file(files):
    # tạo 1 danh sách để chứa các dòng văn bản của những đoạn văn bản có tronh các filé 
    text_list = []
    # duyệt qua các file 
    for _ in files:
        # sử dụng with open để mở các file và gán nó cho f 
        with open(_) as f:
            # Duyệt qua các dòng văn bản có trong file và add vào list 
            for line in f :
                text_list.append(line)
    # trả về danh sách các ròng văn bản 
    return text_list

# Xây dựng hàm đọc và định dạng văn bản từ văn bản thô 
# và trả về 1 bảng dữ liệu có 2 cột là review và sentiment 
def get_data_from_text_files(folder_name):
    # sử dụng biến glob để định dạng toàn bộ văn bản thành đồng dạng 
    # của thư mục folder_name và các tệp này có định dạng  aclImdb/folder_name/pos/*.txt
    pos_files = glob.glob('aclImdb/' + folder_name + '/pos/*.txt')
    # Tiến hành đọc và lấy ra các dòng dữ liệu từ file 
    pos_texts = get_text_list_from_file(pos_files)
    # Thực hiện tương tự với định dạng thư mục aclImdb/folder_name/neg/*.txt 
    neg_files = glob.glob("aclImdb/" + folder_name + "/neg/*.txt")
    neg_texts = get_text_list_from_file(neg_files)

    # Tiến hành xây dựng dataFrame với 2 cột review và sentiment 
    df = pd.DataFrame(
        {
            'review': pos_texts + neg_texts ,# số lượng phần từ trong một cột 
            # với cột sentiment ta biến đổi dạng 0 và 1 
            'sentiment' : [0] * len(pos_texts) + [1] * len(neg_texts)
        }
    )
    # sử dụng hàm samples để xáo trộn mẫu ngẫu nhiên trong tệp df 
    # kết quả alf 1 dataFrame có cùng kích thước với df ban đầu nhưng bị xáo trộn 
    # sau đó dùng hàm reset_index để đặt lại chỉ số index lại từ đầu cho các hàng 
    df = df.sample(len(df)).reset_index(drop=True)
    return df 


# Xây dựng dữ liệu train và tets 
train_df = get_data_from_text_files('train')
test_df = get_data_from_text_files('test')
all_data = train_df.append(test_df)


Dataset preparetion ( Chuẩn bị bộ dữ liệu cho Mô hình)

In [ ]:
# Xây dựng hàm điều chỉnh tiêu chuẩn hóa dưc liệu 
# 1 : định dạnh đồng bộ from chữ 
# 2 : xử dụng biểu thức chính quy loại bỏ định dạng xuống dòng trong HTML 
# 3 : tiến hành loại bỏ và thay thế các ký tự đặc biệt 
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    # loại bỏ định dạng HTML 
    trippedhtml = tf.strings.regex_replace(lowercase , "<br />" ," ")
    # Loại bỏ đi ký tự dặc biệt dùng escape để thoát khỏi ký tự đặc biệt trong 
    # biểu thức chính quy 
    return tf.strings.regex_replace (
        trippedhtml , '[%s]' % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"),""
    )

# Xây dựng hàm thực hiện vector hóa các tokens 

def get_vector_layer(texts , vocab_size , max_seq , special_tokens=['[MASK]']):
    """"
        Xây dựng vector hóa văn bản 

    Args:
        text (list) : List of string i.e input texts 
        vocab_size (int) : Size of vocabulary
        max_seq (int) : Maximum sequence length.
        special_tokens (list , optional) : List of special tokens . Default to ['[MASK]].

    Return :
     layers.Layer : Return TextVectorization Keras Layer

    """
    # Khởi tạo hàm TextVectorization từ keras
    vectorization = keras.layers.TextVectorization(
        # max_tokens  \\ vocab_size  , output frome = int 
        max_tokens=vocab_size , 
        output_mode= 'int',
        standardize= custom_standardization , 
        output_sequence_length=max_seq, 
    )
    # sử dụng hàm adapt để biến đổi phù hợp đầu vào với vectorization
    vectorization.adapt(texts)

    # Xây dựng 1 từ điển bằng cách sử dụng hàm get_vocabulary () và nhận về tần xuất 
    # xuất hiện của các tokens giảm dần 
    vocab = vectorization.get_vocabulary()
    # Tiến hành xây dựng bộ từ vựng bằng cachs bỏ đi 2 phần tử đầu thường là các tokens
    # cls , esp và thay thế các tokens đực biệt = ['mask']
    vocab = vocab[2 : vocab_size - len(special_tokens)] + ['[mask]']
    # cuối cung cập nhật lại bộ từ vựng và trả về nó 
    vectorization.set_vocabulary(vocab)
    return vectorization 

vectorize_layer = get_vector_layer(
    all_data.review.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN,
    special_tokens=["[mask]"],
)

# Nhận mặt nạ id mã thông báo cho mô hình mặt nạ ngôn ngữ
mask_token_id = vectorize_layer(["[mask]"]).numpy()[0][0]

# xây dựng hàm mã hóa dữ liệu 
def encode(texts):
    encoded_texts = vectorize_layer(texts)
    return encoded_texts

# Xây dựng hàm thực hiện che đi 15 % number of tokens và gán nhãn cho chúng 
def get_mask_input_and_labels(encoded_texts):
    # che đi 15 % số tokens 
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.15 
    # Bỏ che những tokens đặc biệt 
    inp_mask[encoded_texts <= 2] = False 
    # Thiết lập bộ nhãn tương ứng = -1
    labels = -1 * np.ones(encoded_texts.shape , dtype=int) 
    # Gán nhãn cho mặt nạ mã thông báo 
    # tất cả các tokens bị tre đi = -1 và còn lại được set bởi chỉ số tokens của encoded_texts
    labels[inp_mask] = encoded_texts[inp_mask]

    # Chuẩn bị một bộ dữ liệu sao chép nguyên bản  
    encoded_texts_masked = np.copy(encoded_texts)
    # Thay thế 90 % các token bị che đi thành tokens mask và giữ lại 10 % 
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) <  0.9)
    # sau đó thay thế các token bị chee mặt nạ từ danh sách encoded_texts_mask
    # bằng Tokens Mask 
    encoded_texts_masked [inp_mask_2mask] = mask_token_id 

    # đặt ra 10 % số tokens còn lại của inp_mask_2mask là các tokens bị che đi không bị thay thế 
    # bởi Tokens Mask thành các tokens ngẫu nhiên khác trừ nhũng token đặc biệt 
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1/9 )
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3 , mask_token_id , inp_mask_2random.sum()
    ) # cuối cùng encoded_texts_mask sẽ chứa 10% tokens ngẫu nhiên và 90% token bị che đi 

    # Chuẩn bị sample_weights chuyển đến phương thức phù hợp 
    # khởi tạo samples = shape(labels) with full 1
    sample_weights = np.ones(labels.shape)
    # che đi các nhãn có giá trị = -1 thay thế = 0 nó thường là các tokens_mask , padded_tokens..
    # để bỏ qua trong quá trình tính toán và giảm thiểu chi phí tính toán 
    sample_weights[labels == -1] = 0

    # Khởi tạo bộ nhãn y_labels bằng chính dữ liệu nguyên bản . 
    y_labels = np.copy(encoded_texts)

    # trả về encoded_texts_masked , y_labels , sample_weight 
    return encoded_texts_masked , y_labels , sample_weights



# Xây dựng các tầng dữ liệu cho các lớp mô hình 

# bộ 25000 mẫu cho huấn luyện mô hình
# phải chuyển đổi x thành các tokens_ids (int) và lấy ra giá trị của cột review  
x_train = encode(train_df.review.values)  # encoded review with vectorizer 
y_train = train_df.sentiment.values 

train_classifier_ds = (
    tf.data.Dataset.from_tensor_slices((x_train , y_train))
    .shuffle(1000)
    .batch(config.BATCH_SIZE)
)

# Và một bộ tương tự với 25 0000 dữ liệu thử nghiệm 
# thực hiện tương tự như bộ dữ liệu huân luyện 
x_test = encode(test_df.review.values)
y_test = test_df.sentiment.values 

test_classifier_df = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(config.BATCH_SIZE)
)

# Build dataset for end to end model input (will be used at the end)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices(
    (test_df.review.values, y_test)
).batch(config.BATCH_SIZE)


# Prepare data for masked language model 
x_all_review = encode(all_data.review.values)
x_masked_train , y_masked_labels , sample_weights = get_mask_input_and_labels(
    x_all_review
)

mlm_ds = tf.data.Dataset.from_tensor_slices(
    (x_masked_train, y_masked_labels, sample_weights)
)
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

Create Bert model (Pretraining Model) ỏ masked language modeling 

In [ ]:
# xây dựng bert module 
def bert_module(q , k , v ,  i):
    # Add multihead Attention 
    attention_output = layers.MultiHeadAttention(
        num_heads=config.NUM_HEADS, 
        key_dim=config.EMBED_DIM , 
        name='encoder_{}multiheadattrntion'.format(i),
    )(q , k , v)

    # add dropout for attention output 
    attention_output = layers.Dropout(
        0.1 ,
        name ='encoder_{}/attn_dropout'.format(i)
    )(attention_output)  
    # add layers Normalization 
    attention_output = layers.LayerNormalization(
        epsilon=1e-6,name='encoder_{}/att_layernormalization'.format(i)
    )(q + attention_output)


    # Implement Feedforward netwwordk 
    ffn = keras.Sequential(
        [
            layers.Dense(units=config.FF_DIM,activation="relu"), 
            layers.Dense(config.EMBED_DIM)
        ], 
        name="encoder_{}/ffn".format(i),
    )
    # add ffn layers 
    ffn_output = ffn(attention_output)
    # add ffn dropout
    ffn_output = layers.Dropout(rate=0.1,name='encoder_{}/ffn_dropout'.format(i))(ffn_output)
    # add output layers = atention_ouput + ffn_output 
    sequence_output = layers.LayerNormalization(
        epsilon=1e-6, name="encoder_{}/ffn_layernormalization".format(i)
    )(attention_output + ffn_output)

    return sequence_output


# Xây dụng hàm tạo ma trận positional encoding 
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            # theo công thức pos / 10000^2 *j // d / emb
            # công thức này là công thức tính toán vị trí trong transformer gốc 
            [pos / np.power(10000, 2 *(j // 2) /d_emb) for j in range(d_emb)    ]
            if pos != 0 
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    # T thay thế các giá trị ở chiều chằn và lẻ 
    # trường hợp chẵn ta sử dụng hàm sin
    pos_enc[1:, 0 :: 2] = np.sin(pos_enc[1:, 0:: 2])
    # Trường hợp lẻ ta sử dụng cos function 
    pos_enc[1:, 1:: 2] = np.cos(pos_enc[1:, 1:: 2])
    # Cuối cùng trả về vị trí cho các tokens 

# xây dựng hàm loss và phương thức loss tracker để cập nhật số liệu 
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE
)
loss_tracker = tf.keras.metrics.Mean(name='loss')


class MaskedLanguageModel(tf.keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 3 :
            features , labels , sample_weight = inputs 
        else : 
            features , labels = inputs
            sample_weight = None

        with tf.GradientTape() as tape:
            # Thực hiện dự đoán 
            predictions = self(features , training=True)
            # Tính toán chi phí 
            loss = loss_fn(labels , predictions , sample_weight=sample_weight)
        
        # Tính toán gradient 
        gradients = tape.gradient(loss , self.trainable_variables) 

        # cập nhật trọng số bằng trình tối ưu hóa
        self.optimizer.apply_gradients(zip(gradients , self.trainable_variables))
        #Tính toán và cập nhật số liệu 
        loss_tracker.update_state(loss, sample_weight=sample_weight)

        # TRả lại tên chỉ số ánh xạ chính tả và chỉ số hiện tại 
        return {'loss': loss_tracker.result()}
    @property 
    def metrics(self):
        return [loss_tracker]
    

# Khởi tạo mô hình bert mặt nạ ngôn ngữ mlm

def create_masked_language_bert_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)

    word_embedding = layers.Embedding(
        config.VOCAB_SIZE , config.EMBED_DIM, name='word_embedding'
    )(inputs)
    position_embedding = layers.Embedding(
        input_dim=config.MAX_LEN, 
        output_dim=config.EMBED_DIM, 
        weights=[get_pos_encoding_matrix[config.MAX_LEN, config.EMBED_DIM]],
        name='position_embedding',
    )(tf.range(start=0 ,limit=config.MAX_LEN, delta=1))
    embeddings = word_embedding + position_embedding

    encoder_output = embeddings
    for i in range(config.NUM_LAYERS):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, i)

    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(
        encoder_output
    )
    mlm_model = MaskedLanguageModel(inputs, mlm_output, name="masked_bert_model")

    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    mlm_model.compile(optimizer=optimizer)
    return mlm_model

# tạo ra 2 bộ từ điển mới và đặt cho bộ từ điển nàylà 1 iterator có thể lặp 
# và tần xuất các từ được xuất hiện giảm dần 

id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y : x for x  , y in id2token.items()}

# Xây dựng lớp maskedGenerator với mục đích để tạo ra các dự đoán cho các tokens 
# bị ẩn trong văn bản 

class MaskedTextGenerator(keras.callbacks.Callback):
    def __init__(self, sample_tokens, top_k =5):
        # sample_tokens là một mảng các tokens được mã hóa bởi vectorize_layers 
        self.sample_tokens = sample_tokens
        # Top k là tham số đại diện cho số lượng tokens có xác xuất cao nhất 
        # được chọn để dự đoán cho token bị che đi 
        self.k = top_k
        
    # Xây dựng phương thưc mã hóa (phương thức mã hóa ngược )
    def decode(self, tokens):
        # tạo ra 1 danh sách các token tương ứng với các mã số trong mảng tokens
        # bỏ qua các mã số = 0 || sử dụng tập từ điển xây dựng trước để ánh xạ 
        # các mã số tokens_id (int) sang các token 
        # sử dụng join để nối các token trong dnah sách thành văn bản 
        return " ". join([id2token[t] for t in tokens if t != 0])
    
    # Xây dựng phương thức chuyển đổi một mã số thành các token tương ứng 
    def convert_ids_to_tokens(self, id):
        # Tham số id là chỉ số index của token trong bộ từ điển 
        # và trả về 1 token tương ứng với vị trí idex trong từ điển 
        return id2token(id)
    
    def on_epoch_end(self, epoch, logs=None):
        # dự đoán tokens thay thế cho câu tar về 1 xác xuất soft max 
        prediction = self.model.predict(self.sample_tokens)

        # Tìm vị trí của tokens bị che đi trong mảng sample_tokens sử dụng hàm where để
        # tìm kiếm chỉ số tokens tương ứng của mask trong ma trận masked_tokens_id
        masked_index = np.where(self.sample_tokens == mask_token_id)
        # sau đó lấy ra chỉ số cột của của tokens bị ẩn trong mảng sample_tokens 
        masked_index = masked_index[1]

        # lấy ra ma trận có xác xuất cao nhất của mỗi tokens bị che giấu 
        # shape = [1, vocabulary] mỗi phần tử trong ma trận là xác xuất của 1 token 
        # có thể được điền vaò vị trí bị che giấu 
        masked_prediction  = prediction[0][masked_index]
        # Tiếp theo sắp xếp các tokens theo thứ tự giảm dần của xác xuất , và lấy ra 
        # tokens có xác xuất cao nhất top.k . Kết quả sẽ được lưu vào hai biến: 
        # top_indices và values. Biến top_indices là một mảng chứa các mã số của các token được chọn,
        # biến values là một mảng chứa các xác suất tương ứng.        
        top_indices = masked_prediction[0].argsort()[-self.k :][:: -1]
        values = masked_prediction[0][top_indices]


        for i in range (len(top_indices)): 
            p = top_indices[i] # lấy ra mã số của token i 
            v = values[i] # lấy ra xác xuất của token i 3
            tokens = np.copy(sample_tokens[0]) 
            # p là mã số theo xác xuất cao nhất của tokens có nghĩa P là biến có khả năng là 
            # token bị che đi 
            #  Đồng thời cũng cho biết vị trí của P trong ma trận chứa tokens bị che đi 
            tokens[masked_index[0]] = p 
            result = {
                "input_text": self.decode(sample_tokens[0].numpy()),
                "prediction": self.decode(tokens),
                "probability": v,
                "predicted mask token": self.convert_ids_to_tokens(p),
            }
            pprint(result)


sample_tokens = vectorize_layer(["I have watched this [mask] and it was awesome"])
generator_callback = MaskedTextGenerator(sample_tokens.numpy())

bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()

7.905882352941177


Train and save 

In [ ]:
bert_masked_model.fit(mlm_ds, epochs=5, callbacks=[generator_callback])
bert_masked_model.save("bert_mlm_imdb.h5")

Fine-tune a sentiment classification model

In [ ]:
# Load pretrained bert model 
mlm_model = keras.models.load_model(
    'bert_mlm_imdb.h5', custom_objects={'MaskedLanguageModel': MaskedLanguageModel}
)

pretrained_bert_model = tf.keras.Model(
    mlm_model.input , mlm_model.get_layer("encoder_0/ffn_layernormalization").output
)
# Freeze it
pretrained_bert_model.trainable = False

def create_classifier_bert_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    sequence_output = pretrained_bert_model(inputs)
    pooled_output = layers.GlobalMaxPooling1D()(sequence_output)
    hidden_layer = layers.Dense(64 , activation='relu')(pooled_output)
    outputs = layers.Dense(1 , activation='sigmoid')(hidden_layer)
    classifier_model = keras.Model(inputs , outputs=outputs ,name='classification')
    optimizer = keras.optimizers.Adam()
    classifier_model.compile(
        optimizer=optimizer , loss='binary_crossentropy' , metrics=['accuracy']
    )
    return classifier_model

classifier_model = create_classifier_bert_model()
classifier_model.summary()


In [ ]:
# Train classifier with fronze bert stage 
classifier_model.fit(
    train_classifier_ds, 
    epochs = 5, 
    validation_batch_size=test_classifier_df
)

# Bỏ đóng băng BERT cho tinh chỉnh mô hình 
pretrained_bert_model.trainable = True 
classifier_model.compile(
    train_classifier_ds , 
    epochs = 5 , 
    validation_data = test_classifier_df, 
)

Create an end-to-end model and evaluate it

In [ ]:
def get_end_to_end(model):
    inputs_string = layers.Input(shape=(1,), stype='string')
    # Thực hiện Bert Tokenizer 
    indices = vectorize_layer(inputs_string)
    outputs = model(indices)
    # End-to-end model 
    end_to_end_model = keras.Model(inputs=inputs_string, outputs = outputs,
                    name='end_to_end_model')
    
    # optimizers 
    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    end_to_end_model.compile(
        optimizer=optimizer , loss= 'binary_crossentropy', 
        metrics=['accuracy']
    )
    return end_to_end_model


end_to_end_classification_model = get_end_to_end(classifier_model)
end_to_end_classification_model.evaluate()
